In [1]:
import tweepy
import pandas as pd
import os
from dotenv import main
import time


In [2]:
def configure():
    main.load_dotenv()
configure()


In [3]:
auth = tweepy.OAuthHandler(os.getenv('API_KEY'), os.getenv('API_KEY_SECRET'))
auth.set_access_token(os.getenv('ACCESS_TOKEN'), os.getenv('ACCESS_TOKEN_SECRET'))
api = tweepy.API(auth)


In [20]:
def get_tweets(query):
    query = query + "-filter:retweets"
    tweets = tweepy.Cursor(api.search_tweets,q=query,lang="en").items(1000)
    tw_copy = []
    for tweet in tweets:
        tw_copy.append(tweet)
    print(f"{len(tw_copy)} tweets found for {query}")
    return tw_copy

In [26]:
def get_tweet_df(species):
    tweets_df = pd.DataFrame()
    for s in species:
        start = time.process_time()
        tweets_copy = get_tweets(s)
        end = time.process_time()
        print(f"Time for keyword {s}: {end-start}")
        # time.sleep(10)
        for tweet in tweets_copy:
            hashtags = []
            try:
                for hashtag in tweet.entities["hashtags"]:
                    hashtags.append(hashtag)
                text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
            except:
                print("Some error getting hashtags")
            tweets_df = pd.concat([tweets_df,pd.DataFrame({
                'user_name': tweet.user.name,
                "user_description": tweet.user.description,
                "user_location": tweet.user.location,
                "user_verification": tweet.user.verified,
                "date": tweet.created_at,
                "text": text,
                "hashtags": [hashtags if hashtags else None],
                "source": tweet.source
                })],axis=0)

            tweets_df = tweets_df.reset_index(drop=True)
        time.sleep(12)
    return tweets_df

In [4]:
def get_tweets_for_keyword(q, max_tweets=1000):
    """This function gets tweets for a particular keyword"""
    created_at_list = []
    id_str_list = []
    text_list = []
    user_id_list = []
    user_name_list = []

    try:
        for tweet in tweepy.Cursor(api.search_tweets,
                                   q=q,
                                   count=100,
                                   result_type="recent",
                                   include_entities=True,
                                   lang="en").items(max_tweets):

            created_at_list.append(tweet.created_at)
            id_str_list.append(tweet.id_str)
            text_list.append(tweet.text)
            user_id_list.append(tweet.user.id_str)
            user_name_list.append(tweet.user.name)


        # Create a Pandas DataFrame from the data.
        df = pd.DataFrame({'created_at': created_at_list,
                                'id': id_str_list, 
                                'text': text_list, 
                                'user_id': user_id_list, 
                                'user_name': user_name_list, 
                                })

        tweets_file_name = "tweets_" + q.replace(" ", "_") + ".csv"
        return df
    except tweepy.errors.TweepyException as err:
        print(err)


In [6]:
CES = [] #critically endangered species
with open('Critically Endangered Species.txt','r') as f:
    l = f.readlines()
    for name in l:
        t = ''.join(i for i in name if not i.isdigit())
        t = t.replace('.','')
        CES.append(t)
f.close()
print(CES[0])


 Sumatran rhinoceros



In [7]:
PES = [] #possibly extinct species
with open('Possibly Extinct.txt','r') as f:
    l = f.readlines()
    for name in l:
        t = ''.join(i for i in name if not i.isdigit())
        t = t.replace('.','')
        PES.append(t)
f.close()
print(PES[0])


 Northern white rhinoceros



In [8]:
VS = [] #vulnerable species
with open('Vulnerable Species.txt','r') as f:
    l = f.readlines()
    for name in l:
        t = ''.join(i for i in name if not i.isdigit())
        t = t.replace('.','')
        VS.append(t)
f.close()
print(VS[0])


 Long-tailed pangolin



In [27]:
Tweets_C = get_tweet_df(CES)

8 tweets found for ﻿ Sumatran rhinoceros
-filter:retweets
Time for keyword ﻿ Sumatran rhinoceros
: 0.06622860500000005
0 tweets found for  Eastern black rhinoceros
-filter:retweets
Time for keyword  Eastern black rhinoceros
: 0.04066524699999974
0 tweets found for  South-central black rhinoceros
-filter:retweets
Time for keyword  South-central black rhinoceros
: 0.041874514999999946
0 tweets found for  African wild donkey
-filter:retweets
Time for keyword  African wild donkey
: 0.03736765399999964
2 tweets found for  Javan rhinoceros
-filter:retweets
Time for keyword  Javan rhinoceros
: 0.08046221199999959
0 tweets found for  Tonkin black crested gibbon
-filter:retweets
Time for keyword  Tonkin black crested gibbon
: 0.024685530000000178
0 tweets found for  West Yunnan black crested gibbon
-filter:retweets
Time for keyword  West Yunnan black crested gibbon
: 0.03460977499999984
0 tweets found for  Central Yunnan black crested gibbon
-filter:retweets
Time for keyword  Central Yunnan bla

In [1]:
Tweets_C

NameError: name 'Tweets_C' is not defined

In [8]:
CES_Tweets = {}
for species in CES:
    start = time.process_time()
    tweets = get_tweets_for_keyword(species,100)
    end = time.process_time()
    print(species,(end-start))
    CES_Tweets[species] = tweets


429 Too Many Requests
88 - Rate limit exceeded
﻿ Sumatran rhinoceros
 0.09311809800000015
429 Too Many Requests
88 - Rate limit exceeded
 Eastern black rhinoceros
 0.036352513999999836
429 Too Many Requests
88 - Rate limit exceeded
 South-central black rhinoceros
 0.03418400599999982
429 Too Many Requests
88 - Rate limit exceeded
 African wild donkey
 0.03551366399999978
429 Too Many Requests
88 - Rate limit exceeded
 Javan rhinoceros
 0.03963500899999994
429 Too Many Requests
88 - Rate limit exceeded
 Tonkin black crested gibbon
 0.03715217400000004
429 Too Many Requests
88 - Rate limit exceeded
 West Yunnan black crested gibbon
 0.03909554400000026
429 Too Many Requests
88 - Rate limit exceeded
 Central Yunnan black crested gibbon
 0.04204248200000027
429 Too Many Requests
88 - Rate limit exceeded
 Laotian black crested gibbon
 0.04286872200000014
429 Too Many Requests
88 - Rate limit exceeded
 Hainan black crested gibbon
 0.04011643799999964
429 Too Many Requests
88 - Rate limit exc

In [9]:
print(CES_Tweets)

{'\ufeff Sumatran rhinoceros\n': None, ' Eastern black rhinoceros\n': None, ' South-central black rhinoceros\n': None, ' African wild donkey\n': None, ' Javan rhinoceros\n': None, ' Tonkin black crested gibbon\n': None, ' West Yunnan black crested gibbon\n': None, ' Central Yunnan black crested gibbon\n': None, ' Laotian black crested gibbon\n': None, ' Hainan black crested gibbon\n': None, ' Northern white-cheeked gibbon\n': None, ' Eastern black crested gibbon\n': None, ' Eastern lowland gorilla\n': None, ' Cross River gorilla\n': None, ' Western lowland gorilla\n': None, ' Sumatran orangutan\n': None, ' Bornean orangutan\n': None, ' Northeast Bornean orangutan\n': None, ' Northwest Bornean orangutan\n': None, ' Southwest Bornean orangutan\n': None, ' Tapanuli orangutan\n': None, " Sibree's dwarf lemur\n": None, ' Gray-headed lemur\n': None, ' Blue-eyed black lemur\n': None, ' Mongoose lemur\n': None, ' Lac Alaotra bamboo lemur\n': None, ' Golden bamboo lemur\n': None, ' Indri\n': No

In [10]:
import pickle

In [ ]:
ces_file = open('CES_data.pkl',"wb")
pickle.dump(CES_Tweets,ces_file)
ces_file.close()